# 🛡️ Interactive Squad Simulation
Follow the directions at the top of each cell. After running the last cell, use the sliders and dropdowns below to run and visualize the simulation. (Test comment)

In [1]:
# If you are working in colab, use this block to clone the GitHub repo and install dependencies.
%cd /content/
!rm -rf SE3250-Spring2025-SquadSimulation
!git clone https://github.com/SuprMunchkin/SE3250-Spring2025-SquadSimulation.git
%cd SE3250-Spring2025-SquadSimulation
!pip install -r requirements.txt
%ls

/content
Cloning into 'SE3250-Spring2025-SquadSimulation'...
remote: Enumerating objects: 140, done.
remote: Counting objects: 100% (140/140), done.
remote: Compressing objects: 100% (97/97), done.
remote: Total 140 (delta 54), reused 98 (delta 28), pack-reused 0 (from 0)
Receiving objects: 100% (140/140), 1.79 MiB | 4.23 MiB/s, done.
Resolving deltas: 100% (54/54), done.
/content/SE3250-Spring2025-SquadSimulation
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 717.0/717.0 kB 21.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.2/150.2 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 54.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 55.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.4/48.4 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 886.0/886.0 kB 39.4 MB/

In [2]:
# If you are working in colab, run this codeblock to change branches or pull down the latest changes.
# Be sure to change the branch name to the one you are working on!...
!git fetch
!git checkout Justin
!git pull
%ls

Branch 'Justin' set up to track remote branch 'Justin' from 'origin'.
Switched to a new branch 'Justin'
Already up to date.
app.py   models/       requirements.txt       statdiagram.py
config/  __pycache__/  squad_simulator.ipynb  view/


In [12]:
# Import and setup
import sys
import yaml
import matplotlib.pyplot as plt
from ipywidgets import interact

import os
sys.path.append("../models")
from models.squad_simulation import run_simulation

# Define interactive runner
def run_interactive_sim(blue_stock, hostile_stock, direction_deviation, armor_type, environment):
    params = {
        "blue_stock": blue_stock,
        "hostile_stock": hostile_stock,
        "direction_deviation": direction_deviation,
        "armor_type": armor_type,
        "environment": environment
    }
    result = run_simulation(params)

    positions = result['positions']
    hostile_positions = result['hostile_positions']
    x_vals, y_vals = zip(*positions)
    hx_vals, hy_vals = zip(*hostile_positions)

    plt.figure(figsize=(8, 6))
    plt.plot(x_vals, y_vals, label='Blue Patrol Path', color='blue')
    plt.plot(hx_vals, hy_vals, label='Hostile Path', linestyle='--', color='red')
    plt.scatter(x_vals[0], y_vals[0], c='green', label='Start', zorder=5)
    plt.scatter(x_vals[-1], y_vals[-1], c='purple', label='End', zorder=5)
    plt.title("Squad Movement Simulation")
    plt.xlabel("X Position")
    plt.ylabel("Y Position")
    plt.legend()
    plt.grid(True)
    plt.axis('equal')
    plt.show()

    print(f"👥 Blue Remaining: {result['blue']['stock']} / {params['blue_stock']}")
    print(f"🔴 Hostile Remaining: {result['hostile']['stock']} / {params['hostile_stock']}")

# Load config
yaml_path = os.path.join(os.path.dirname(__file__), "../config/simulation.yaml")
with open(yaml_path, "r") as f:
    config = yaml.safe_load(f)

# Create interactive widget interface
interact(
    run_interactive_sim,
    blue_stock=(1, 20),
    hostile_stock=(1, 40),
    direction_deviation=(0, 45, 5),
    armor_type=list(config['armor_profiles'].keys()),
    environment=list(config['threat_probs'].keys())
)

NameError: name '__file__' is not defined

In [6]:
%cat ./models/squad_simulation.py

import numpy as np
import pandas as pd
from math import cos, sin, radians, exp
import yaml
import os
import sys

yaml_path = os.path.join(os.path.dirname(__file__), "../config/simulation.yaml")
with open(yaml_path, "r") as f:
    config = yaml.safe_load(f)

threat_library = config["threat_library"]
armor_profiles = config["armor_profiles"]
threat_probs = config["threat_probs"]
fire_rates = config["fire_rates"]


def get_velocity(threat, distance):
    c1, c2, c3 = threat_library[threat]
    return c1 * distance**2 + c2 * distance + c3

def get_defeat_probability(armor, threat, velocity):
    beta0, beta1 = armor_profiles[armor][threat]
    exponent = beta0 + velocity * beta1
    return np.exp(exponent) / (1 + np.exp(exponent))

def attack(blue_patrol, hostile_patrol, env, armor, fire_rates ,distance):
    if distance > 1000:
        return 0, 0
    prob_attack = min(1, 100 / distance if distance > 0 else 1)
    if np.random.random() > prob_attack:
        return 0, 0

    blue_min_fire